In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data/data.csv', sep=';')

In [3]:
df

,ID истории болезни,Осн. диаг. при выписке МКБ10 (текст),Заголовок документа,Кол. лаб. показатель,Значение кол. показателя,Ед. изм. кол. показателя,Норма кол. показателя,Флаг нормы кол. показателя,Кач. лаб. показатель,Значение кач. показателя,Норма кач. показателя,Пол,Дата рождения пациента
0,2e1d0b3f-488a-11ed-ab5a-0050568844e6,Плазмоклеточный лейкоз,Общий анализ крови,Лейкоциты (WBC),1.80,10*9/л,4:11,Пониж,NaN,NaN,NaN,Ж,1962-09-22 00:00:00
1,2e1d0b3f-488a-11ed-ab5a-0050568844e6,Плазмоклеточный лейкоз,Общий анализ крови,Эритроциты (RBC),4.18,10*12/л,"3,8:5,7",Норм,NaN,NaN,NaN,Ж,1962-09-22 00:00:00
2,2e1d0b3f-488a-11ed-ab5a-0050568844e6,Плазмоклеточный лейкоз,Общий анализ крови,Гемоглобин (HGB),120.00,г/л,117:160,Норм,NaN,NaN,NaN,Ж,1962-09-22 00:00:00
3,2e1d0b3f-488a-11ed-ab5a-0050568844e6,Плазмоклеточный лейкоз,Общий анализ крови,Гематокрит (HCT),35.10,%,35:47,Норм,NaN,NaN,NaN,Ж,1962-09-22 00:00:00
4,2e1d0b3f-488a-11ed-ab5a-0050568844e6,Плазмоклеточный лейкоз,Общий анализ крови,Средний объем эритроцита (MCV),84.00,фл,80:99,Норм,NaN,NaN,NaN,Ж,1962-09-22 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
36292,516f652e-8bb8-11eb-bb95-2cea7fe73f75,Острый лейкоз неуточненного клеточного типа,Общий анализ крови,Базофилы,1.00,%,0:2,Норм,Комментарий,Тромбоциты проверены. Рекомендована консультац...,_,М,2014-10-17 00:00:00
36293,516f652e-8bb8-11eb-bb95-2cea7fe73f75,Острый лейкоз неуточненного клеточного типа,Общий анализ крови,Эозинофилы,0.00,%,0:5,Норм,Комментарий,Тромбоциты проверены. Рекомендована консультац...,_,М,2014-10-17 00:00:00
36294,516f652e-8bb8-11eb-bb95-2cea7fe73f75,Острый лейкоз неуточненного клеточного типа,Общий анализ крови,Пролимфоциты,0.00,%,0:0,Норм,Комментарий,Тромбоциты проверены. Рекомендована консультац...,_,М,2014-10-17 00:00:00
36295,516f652e-8bb8-11eb-bb95-2cea7fe73f75,Острый лейкоз неуточненного клеточного типа,Общий анализ крови,Лимфоциты,65.00,%,18:44,Повыш,Комментарий,Тромбоциты проверены. Рекомендована консультац...,_,М,2014-10-17 00:00:00


In [4]:
df['Значение кач. показателя'].unique()

array([nan, 'плазматические клетки с омоложением ядра',
       'из 34 моноцитоидных элементов 9 промоноцитов',
       'микроагрегаты тромбоцитов отсутствуют. тромбоциты при подсчете по мазку - 75х10*9/л',
       'из 14 моноцитоидных элементов 8 промоноциты, тромбоциты по мазку  соответствуют счету анализатора.',
       'микроагрегаты тромбоцитов не обнаружены. тромбоциты при подсчете по мазку - 51х10*9/л',
       '+', '++', 'мишеневидные эритроциты',
       'встречаются плазматические клетки с омоложенными ядрами',
       'сгусток крови!', 'тромбоциты по мазку 62х10*9/л,', 'Сгусток',
       'из 24 моноцитоидных элементов 16 промоноцитов', 'проверено',
       'тромбоциты проверены по мазку', 'сгусток крови',
       'клетки лейколиза в умеренном количестве',
       'встречаются микросгустки тромбоцитов',
       'микроагрегаты тромбоцитов отсутствуют, тромбоциты при подсчете по мазку - 70х10*9/л',
       'тромбоциты по мазку 81х10*9/л',
       'встречаются полихроматофильные эритроциты.',